# 0. Imports

In [3]:
# for data handling and manipulation:
import pandas as pd
import numpy as np

# for reading image dataset
import cv2

# for os handling
import os

# for dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA


# 1. Reading image data for accidents

In [2]:
# defining the path for accident images:
path_for_accident_images = "/kaggle/input/accident-detection-from-cctv-footage/data/train/Accident"  

# getting  list of all images in the accident images available in the dataset
accident_images = os.listdir(path_for_accident_images)

accident_images_paths = ["/kaggle/input/accident-detection-from-cctv-footage/data/train/Accident/" + i for i in accident_images]


# Viewing images

In [3]:
# Code for seeing the training data images
"""
for file in accident_images:
    # cv2 requires the path of the image for reading it:
    img_path = os.path.join(path_for_accident_images,file)
    
    # reading the image using it's path:
    image = cv2.imread(img_path)

    # showing the image
    cv2.imshow('Image', image)

    # making cv2 wait before closing the image, for us to press any key on the keyboard
    cv2.waitKey(0)
    
# destroying all windows in the end: 
cv2.destroyAllWindows()

"""

"\nfor file in accident_images:\n    # cv2 requires the path of the image for reading it:\n    img_path = os.path.join(path_for_accident_images,file)\n    \n    # reading the image using it's path:\n    image = cv2.imread(img_path)\n\n    # showing the image\n    cv2.imshow('Image', image)\n\n    # making cv2 wait before closing the image, for us to press any key on the keyboard\n    cv2.waitKey(0)\n    \n# destroying all windows in the end: \ncv2.destroyAllWindows()\n\n"

In [4]:
# Code for seeing the training data images if converted to grayscale
""""
for file in accident_images:
    # cv2 requires the path of the image for reading it:
    img_path = os.path.join(path_for_accident_images,file)
    
    # reading the image using it's path:
    image = cv2.imread(img_path)
    
    # converting the RGB image to Grayscale:
    gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

    # showing the image
    cv2.imshow('Image', gray_image)

    # making cv2 wait before closing the image, for us to press any key on the keyboard
    cv2.waitKey(0)
    
# destroying all windows in the end: 
cv2.destroyAllWindows()

"""

'"\nfor file in accident_images:\n    # cv2 requires the path of the image for reading it:\n    img_path = os.path.join(path_for_accident_images,file)\n    \n    # reading the image using it\'s path:\n    image = cv2.imread(img_path)\n    \n    # converting the RGB image to Grayscale:\n    gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)\n\n    # showing the image\n    cv2.imshow(\'Image\', gray_image)\n\n    # making cv2 wait before closing the image, for us to press any key on the keyboard\n    cv2.waitKey(0)\n    \n# destroying all windows in the end: \ncv2.destroyAllWindows()\n\n'

In [5]:
# Code for checking the dimensions of the image data

for file in accident_images:
    # cv2 requires the path of the image for reading it:
    img_path = os.path.join(path_for_accident_images,file)
    
    # reading the image using it's path:
    image = cv2.imread(img_path)
    print(image.shape)

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(576, 720, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280,

### It turns out, the images have different dimensions, we need to resize them to a uniform dimension, because we have to flatten them into input features for each image eventually, and images can't have different number of input features associated with them

### We are going to upscale the smaller images using Bicubic Interpolation, because downsizing images causes a lot of loss of precious pixel data

In [6]:
# Code for resizing the smaller images and storing all the images

gray_images = []
color_images = []

for file in accident_images_paths:

    # reading the image using it's path:
    color_image = cv2.imread(file)

    # also converting it to grayscale to reduce the number of features
    gray_image = cv2.cvtColor(color_image,cv2.COLOR_BGR2GRAY)
    
    if color_image.shape != (720,1280,3) or gray_image.shape != (720,1280):
        
        new_height = 720
        new_width = 1280
        
        # resizing the images
        color_resized_image = cv2.resize(color_image, (new_height, new_width), interpolation = cv2.INTER_CUBIC)
        gray_resized_image = cv2.resize(gray_image, (new_height, new_width), interpolation = cv2.INTER_CUBIC)
    else:
        color_resized_image = color_image
        gray_resized_image = gray_image
        
    # flattening the image
    final_gray_image = gray_resized_image.flatten()
    final_color_image = color_resized_image.flatten()
 
    gray_images.append(final_gray_image)
    color_images.append(final_color_image)

In [7]:
# Code for checking the dimensions of the image data

for image in gray_images:
    
    # print the shape of the image
    if image.shape[0] != new_height*new_width:
        print(image.shape)


print('#'*20)

for image in color_images:
    
    # print the shape of the image
    if image.shape[0] != new_height*new_width*3:
        print(image.shape)

####################


### Hence, all images are now of the same dimension : (2764800,) : Means 1-D with 2764800 elements for colored images and 921600 for gray images

In [8]:
len(gray_images), len(color_images)   # as expected 369 accident images for training

(369, 369)

In [9]:
# creating the corresponding y_labels
y = [1]*len(accident_images)

In [10]:
y

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


# 2. Reading image data for non-accidents

In [11]:
# defining the path for accident images:
path_for_non_accident_images = "/kaggle/input/accident-detection-from-cctv-footage/data/train/Non Accident"

# getting  list of all images in the accident images available in the dataset
non_accident_images = os.listdir(path_for_non_accident_images)

non_accident_images_paths = ["/kaggle/input/accident-detection-from-cctv-footage/data/train/Non Accident/" + i for i in non_accident_images]

In [12]:
# Code for checking the dimensions of the image data

for file in non_accident_images_paths:
    
    # reading the image using it's path:
    image = cv2.imread(file)
    
    # print the dimensions of the image
    print(image.shape)
    

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(576, 720, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280,

### Here too, we can see, some images in the beginning are smaller in size

In [13]:
# Code for resizing the smaller images and storing all the images


for file in non_accident_images_paths:
    
    # reading the image using it's path:
    color_image = cv2.imread(file)

    # also converting it to grayscale to reduce the number of features
    gray_image = cv2.cvtColor(color_image,cv2.COLOR_BGR2GRAY)
    
    if color_image.shape != (720,1280,3) or gray_image.shape != (720,1280):
        
        new_height = 720
        new_width = 1280
        
        # resizing the images
        color_resized_image = cv2.resize(color_image, (new_height, new_width), interpolation = cv2.INTER_CUBIC)
        gray_resized_image = cv2.resize(gray_image, (new_height, new_width), interpolation = cv2.INTER_CUBIC)
    else:
        color_resized_image = color_image
        gray_resized_image = gray_image   
    
    # flattening the image
    final_gray_image = gray_resized_image.flatten()
    final_color_image = color_resized_image.flatten()
   
    gray_images.append(final_gray_image)
    color_images.append(final_color_image)


In [14]:
# Code for checking the dimensions of the image data

for image in gray_images:
    
    # print the shape of the image
    if image.shape[0] != new_height*new_width:
        print(image.shape)


print('#'*20)

for image in color_images:
    
    # print the shape of the image
    if image.shape[0] != new_height*new_width*3:
      print(image.shape)

####################


In [15]:
len(gray_images), len(color_images)   # as expected: accident + non_accident images = 369 + 422 = 791

(791, 791)

In [16]:
# extending y to add labels for the images added to `images`
y.extend([0]*len(non_accident_images))

In [17]:
y

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [18]:
# converting the list of images to a numpy array
color_images = np.array(color_images)

gray_images = np.array(gray_images)

In [19]:
# converting labels array to numpy as well:
y = np.array(y)

### So, now our `gray images` contains the images data converted to grayscale and `color images` contains image data for all colored images, for all accident and non-accident images, flattened into it's rows and `y` contains the corresponding labels for these images.

### 0: Non_accident image, 1: Accident image

### Thankfully, only a few images were smaller in size than the other images, hence, we had to interpolate pixels and make only a few images larger than they originally were

# 3. Image Pre-processing 

Not aware of any techniques as of now.

## 3.1 Dimensionality Reduction  

In [20]:
gray_images 

array([[169, 171, 172, ...,  84,  85,  85],
       [ 81,  62,  20, ...,  51,  52,  52],
       [ 86,  90,  83, ..., 103, 103, 103],
       ...,
       [173, 174, 170, ...,  93,  92,  93],
       [ 65,  68,  64, ...,  25,  27,  29],
       [ 58,  78,  85, ..., 101, 101, 101]], dtype=uint8)

In [21]:
color_images

array([[174, 174, 156, ...,  90,  85,  84],
       [ 93,  91,  57, ...,  20,  65,  38],
       [ 72,  77, 108, ..., 110, 105,  96],
       ...,
       [184, 174, 167, ..., 102,  91,  94],
       [ 72,  60,  72, ...,  17,  31,  30],
       [ 65,  56,  59, ..., 105, 102,  98]], dtype=uint8)

In [22]:
gray_images.shape, color_images.shape

((791, 921600), (791, 2764800))

### Couldn't run PCA on colored images without encountering memory error

### Trying to retain 95% of the variation of the image data

In [23]:
"""
pca = PCA(n_components = 0.95)
X_train = pca.fit_transform(gray_images)

X_train.shape
"""

'\npca = PCA(n_components = 0.95)\nX_train = pca.fit_transform(gray_images)\n\nX_train.shape\n'

### Can retain a maximum of 98 features, because features in train, test and val have to be equal and <= min(samples, features) of train, test and val

In [24]:
pca_gray = PCA(n_components = 98)
X_train_gray = pca_gray.fit_transform(gray_images)

In [25]:
sum(pca_gray.explained_variance_ratio_)

0.9292593067123059

In [26]:
pca_color = PCA(n_components = 98)
X_train_color = pca_color.fit_transform(color_images)

In [27]:
sum(pca_color.explained_variance_ratio_)

0.9312746419282848

In [28]:
X_train_gray

array([[-12387.48180764,  19559.86453961, -29390.00937941, ...,
          -241.11917869,   -659.73537099,     78.59143171],
       [ 22399.09574135,   2037.98593839,  16219.81113874, ...,
          -107.45869357,    132.91009494,     53.0107237 ],
       [ -5646.40808565,  40678.56124055, -11914.08422211, ...,
           775.28970294,    124.72548977,  -1796.03317354],
       ...,
       [    91.27973281,  -9431.64303006,  12869.39982115, ...,
          1093.3746155 ,   -809.75672208,  -1536.37282289],
       [ -3122.27585681,   1590.58283503,   4102.97630181, ...,
          -318.24861263,   -754.89811785,   -797.17956191],
       [ -6839.49068472, -15148.62082936,   2621.33031367, ...,
          -540.79295623,    244.59213871,  -1554.48627677]])

In [29]:
X_train_gray.shape

(791, 98)

In [30]:
X_train_color

array([[-1.48807674e+04,  3.37702825e+04, -5.31024815e+04, ...,
        -7.47567203e+02,  5.87897070e+02, -8.00290756e+02],
       [ 4.18582083e+04,  4.95716525e+03,  3.36635969e+04, ...,
        -2.59856132e+02,  1.35249325e+02,  4.25838921e+01],
       [-4.46680240e+03,  6.65670682e+04, -2.08697992e+04, ...,
        -1.62509969e+03,  3.32420358e+03, -3.63190540e+03],
       ...,
       [-2.82446173e+03, -1.79458451e+04,  1.98483342e+04, ...,
        -3.55968070e+02,  1.81237722e+03, -2.65393149e+02],
       [-6.24969305e+03,  3.62751822e+03,  7.12923753e+03, ...,
        -2.78458102e+03, -6.20031916e+02,  9.68769106e+02],
       [-1.40664790e+04, -2.51845757e+04,  6.43196741e+03, ...,
         4.81291406e+02,  4.91481336e+02,  1.50974867e+03]])

In [31]:
X_train_color.shape

(791, 98)

### Moving on to the task of preparing validation data now after saving the train data

In [32]:
pd.DataFrame(X_train_gray).to_csv('X_train_gray.csv')

In [33]:
pd.DataFrame(X_train_color).to_csv('X_train_color.csv')

In [34]:
pd.DataFrame(y).to_csv('y_train.csv')

# 4. Reading image data for accidents (Validation set)

In [35]:
# defining the path for accident images:
path_for_accident_images = "/kaggle/input/accident-detection-from-cctv-footage/data/val/Accident"  

# getting  list of all images in the accident images available in the dataset
accident_images = os.listdir(path_for_accident_images)

accident_images_paths = ["/kaggle/input/accident-detection-from-cctv-footage/data/val/Accident/" + i for i in accident_images]


In [36]:
# Code for checking the dimensions of the image data

for file in accident_images_paths:

    # reading the image using it's path:
    image = cv2.imread(file)
    print(image.shape)

(576, 720, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(576, 720, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)


In [37]:
# Code for resizing the smaller images and storing all the images

gray_images = []
color_images = []

for file in accident_images:

    # cv2 requires the path of the image for reading it:
    img_path = os.path.join(path_for_accident_images,file)
    
    # reading the image using it's path:
    color_image = cv2.imread(img_path)

    # also converting it to grayscale to reduce the number of features
    gray_image = cv2.cvtColor(color_image,cv2.COLOR_BGR2GRAY)
    
    if color_image.shape != (720,1280,3) or gray_image.shape != (720,1280):
        
        new_height = 720
        new_width = 1280
        
        # resizing the images
        color_resized_image = cv2.resize(color_image, (new_height, new_width), interpolation = cv2.INTER_CUBIC)
        gray_resized_image = cv2.resize(gray_image, (new_height, new_width), interpolation = cv2.INTER_CUBIC)
    
    else:
        color_resized_image = color_image
        gray_resized_image = gray_image    
    
    # flattening the image
    final_gray_image = gray_resized_image.flatten().T
    final_color_image = color_resized_image.flatten().T
    
    gray_images.append(final_gray_image)
    color_images.append(final_color_image)


In [38]:
# Code for checking the dimensions of the image data

for image in gray_images:
    
    # print the shape of the image
    if image.shape[0] != new_height*new_width:
        print(image.shape)


print('#'*20)

for image in color_images:
    
    # print the shape of the image
    if image.shape[0] != new_height*new_width*3:
        print(image.shape)

####################


In [39]:
len(gray_images), len(color_images) 

(46, 46)

In [40]:
y = [1]*len(accident_images)

In [41]:
y

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

# 5. Reading image data for non-accidents (Validation set)

In [42]:
# defining the path for accident images:
path_for_non_accident_images = "/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident"

# getting  list of all images in the accident images available in the dataset
non_accident_images = os.listdir(path_for_non_accident_images)

non_accident_images_paths = ["/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident/" + i for i in non_accident_images]


In [43]:
# Code for checking the dimensions of the image data

for file in non_accident_images_paths:
    
    # reading the image using it's path:
    image = cv2.imread(file)
    
    # print the dimensions of the image
    print(image.shape)

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(576, 720, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(576, 720, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(576, 720, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)


In [44]:
# Code for resizing the smaller images and storing all the images


for file in non_accident_images:

    # cv2 requires the path of the image for reading it:
    img_path = os.path.join(path_for_non_accident_images,file)
    
    # reading the image using it's path:
    color_image = cv2.imread(img_path)

    # also converting it to grayscale to reduce the number of features
    gray_image = cv2.cvtColor(color_image,cv2.COLOR_BGR2GRAY)
    
    if color_image.shape != (720,1280,3) or gray_image.shape != (720,1280,3):
        
        new_height = 720
        new_width = 1280
        
        # resizing the images
        color_resized_image = cv2.resize(color_image, (new_height, new_width), interpolation = cv2.INTER_CUBIC)
        gray_resized_image = cv2.resize(gray_image, (new_height, new_width), interpolation = cv2.INTER_CUBIC)
    
    else:
        color_resized_image = color_image
        gray_resized_image = gray_image
    
    # flattening the image
    final_gray_image = gray_resized_image.flatten()
    final_color_image = color_resized_image.flatten()
    
    gray_images.append(final_gray_image)
    color_images.append(final_color_image)


In [45]:
# Code for checking the dimensions of the image data

for image in gray_images:
    
    # print the shape of the image
    if image.shape[0] != new_height*new_width:
        print(image.shape)


print('#'*20)

for image in color_images:
    
    # print the shape of the image
    if image.shape[0] != new_height*new_width*3:
        print(image.shape)

####################


In [46]:
len(gray_images), len(color_images)

(98, 98)

In [47]:
# extending y to add labels for the images added to `images`
y.extend([0]*len(non_accident_images))

In [48]:
y

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [49]:
# converting the list of images to a numpy array
color_images = np.array(color_images)

gray_images = np.array(gray_images)

In [50]:
# converting labels array to numpy as well:
y = np.array(y)

## Applying PCA to reduce dimension

In [51]:
# To retain 98 features, to match the no. of features of the training set
X_val_gray = pca_gray.transform(gray_images)

In [52]:
X_val_color = pca_color.transform(color_images)

In [53]:
X_val_gray.shape, X_val_color.shape

((98, 98), (98, 98))

In [54]:
pd.DataFrame(X_val_gray).to_csv('X_val_gray.csv')

In [55]:
pd.DataFrame(X_val_color).to_csv('X_val_color.csv')

In [56]:
pd.DataFrame(y).to_csv('y_val.csv')

### Moving on to the task of preparing test data now

# 7. Reading image data for accidents (Test set)

In [9]:
# defining the path for accident images:
path_for_accident_images = "/kaggle/input/accident-detection-from-cctv-footage/data/test/Accident" 

# getting  list of all images in the accident images available in the dataset
accident_images = os.listdir(path_for_accident_images)

accident_images_paths = ["/kaggle/input/accident-detection-from-cctv-footage/data/test/Accident/" + i for i in accident_images]


In [10]:
# Code for checking the dimensions of the image data

for file in accident_images_paths:
    
    # reading the image using it's path:
    image = cv2.imread(file)
    print(image.shape)

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(576, 720, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)


In [11]:
# Code for resizing the smaller images and storing all the images

gray_images = []
color_images = []

for file in accident_images_paths:

    # reading the image using it's path:
    color_image = cv2.imread(file)

    # also converting it to grayscale to reduce the number of features
    gray_image = cv2.cvtColor(color_image,cv2.COLOR_BGR2GRAY)
    
    if color_image.shape != (720,1280,3) or gray_image.shape != (720,1280):
        
        new_height = 720
        new_width = 1280
        
        # resizing the images
        color_resized_image = cv2.resize(color_image, (new_height, new_width), interpolation = cv2.INTER_CUBIC)
        gray_resized_image = cv2.resize(gray_image, (new_height, new_width), interpolation = cv2.INTER_CUBIC)
    
    else:
        color_resized_image = color_image
        gray_resized_image = gray_image
    
    # flattening the image
    final_gray_image = gray_resized_image.flatten().T
    final_color_image = color_resized_image.flatten().T
    
    gray_images.append(final_gray_image)
    color_images.append(final_color_image)


In [12]:
# Code for checking the dimensions of the image data

for image in gray_images:
    
    # print the shape of the image
    if image.shape[0] != new_height*new_width:
        print(image.shape)


print('#'*20)

for image in color_images:
    
    # print the shape of the image
    if image.shape[0] != new_height*new_width*3:
        print(image.shape)

####################


In [13]:
len(gray_images), len(color_images) 

(47, 47)

In [14]:
y = [1]*len(accident_images)

In [15]:
y

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

# 7. Reading image data for non-accidents (Test set)

In [16]:
# defining the path for accident images:
path_for_non_accident_images = "/kaggle/input/accident-detection-from-cctv-footage/data/test/Non Accident" 

# getting  list of all images in the accident images available in the dataset
non_accident_images = os.listdir(path_for_non_accident_images)

non_accident_images_paths = ["/kaggle/input/accident-detection-from-cctv-footage/data/test/Non Accident/" + i for i in non_accident_images]


In [17]:
non_accident_images_paths.remove('/kaggle/input/accident-detection-from-cctv-footage/data/test/Non Accident/Pictures - Shortcut.lnk')

In [18]:
# Code for checking the dimensions of the image data

for file in non_accident_images_paths:
   
    # reading the image using it's path:
    image = cv2.imread(file)
    
    # print the dimensions of the image
    print(image.shape)

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(576, 720, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)


In [19]:
# Code for resizing the smaller images and storing all the images

for file in non_accident_images_paths:

    # reading the image using it's path:
    color_image = cv2.imread(file)

    # also converting it to grayscale to reduce the number of features
    gray_image = cv2.cvtColor(color_image,cv2.COLOR_BGR2GRAY)
    
    if color_image.shape != (720,1280,3) or gray_image.shape != (720,1280):
        
        new_height = 720
        new_width = 1280
        
        # resizing the images
        color_resized_image = cv2.resize(color_image, (new_height, new_width), interpolation = cv2.INTER_CUBIC)
        gray_resized_image = cv2.resize(gray_image, (new_height, new_width), interpolation = cv2.INTER_CUBIC)
    
    else:
        color_resized_image = color_image
        gray_resized_image = gray_image
    
    # flattening the image
    final_gray_image = gray_resized_image.flatten()
    final_color_image = color_resized_image.flatten()
    
    gray_images.append(final_gray_image)
    color_images.append(final_color_image)


In [20]:
# Code for checking the dimensions of the image data

for image in gray_images:
    
    # print the shape of the image
    if image.shape[0] != new_height*new_width:
        print(image.shape)


print('#'*20)

for image in color_images:
    
    # print the shape of the image
    if image.shape[0] != new_height*new_width*3:
        print(image.shape)

####################


In [21]:
len(gray_images), len(color_images)

(100, 100)

In [22]:
# extending y to add labels for the images added to `images`
y.extend([0]*len(non_accident_images_paths))

In [23]:
# converting the list of images to a numpy array
color_images = np.array(color_images)

gray_images = np.array(gray_images)

In [24]:
# converting labels array to numpy as well:
y = np.array(y)

In [25]:
# To retain 98 features, to match the no. of features of the training set
X_test_gray = pca_gray.transform(gray_images)

NameError: name 'pca_gray' is not defined

In [74]:
X_test_color = pca_color.transform(color_images)

In [75]:
X_test_gray.shape, X_test_color.shape

((100, 98), (100, 98))

In [76]:
pd.DataFrame(X_test_gray).to_csv('X_test_gray.csv')

In [77]:
pd.DataFrame(X_test_color).to_csv('X_test_color.csv')

In [26]:
pd.DataFrame(y).to_csv('y_test.csv')

In [ ]:
t1 = pd.read_csv('X_train_gray.csv').drop(columns = ['Unnamed: 0'], axis = 1)
t1.head()

In [ ]:
t2 = pd.read_csv('X_val_gray.csv').drop(columns = ['Unnamed: 0'], axis = 1)
t2.head()

In [ ]:
t3 = pd.concat([t1,t2], axis = 0)
t3

In [ ]:
t4 = pd.read_csv('y_train.csv').drop(columns = ['Unnamed: 0'], axis = 1)
t5 = pd.read_csv('y_val.csv').drop(columns = ['Unnamed: 0'], axis = 1)

t6 = pd.concat([t4,t5], axis = 0)
t6.columns = ['label']
t6

In [ ]:
pd.concat([t3,t6], axis = 1).to_csv('X_train_gray_final.csv')